# Continuous Time Sphere Model

We model the example of short-lived temperature sensors on the Earth in continuous time, using a combination of a queueing model and point process. Throughout the setup of the model, fix a time $T;$ for each time $t \in [0,T]$ the set of sensors $S_t$ at time $t$ is a set of locations on $\mathbb{S}^2,$ where $|S_t|$ is a random variable, and $S_t(i),$ the location of the $i$th sensor is uniformly sampled from the uniform measure on the sphere.

Perhaps the easiest way to set up the model is via a (FIFO) queueing model, or birth-death process representing the sensors:

- We start with $L_0$ number of sensors, each sampled uniformly from the sphere. 
- Sensors are added to the queue every so often, again sampled uniformly from the sphere. 
- Independently of the creation of new sensors, sensors break down at a perhaps different rate, and leave the queue.

We can model the time between consecutive joining/ exiting of the queue by exponential distributions.


## Queueing Model Implementation
Consider a queueing model whose initial length is $L_0,$ the number of initial sensors we begin with. We assume that the queue processes sensors (physically: sensors break down) in a random time $\sim \text{Exp}(\lambda_1),$ and sensors join the queue with interarrival times distributed according to $\text{Exp}(\lambda_2),$ which are independent of the times it takes for the queue to process the input.

We can simulate this queue in the following way. For a given fixed time $t \in [0,T],$ by the memoryless property of exponential distributions, the time until a sensor leaves the queue is still $X_1 \sim \text{Exp}(\lambda_1)$ and the interarrival time is still distributed according to $X_2 \text{Exp}(\lambda_2).$ Now, the time until one of these two actions occurs is a random variable $Z = \min(X_1,X_2) \sim \text{Exp}(\lambda_1+\lambda_2)$ (which can be checked). Now the probability that a sensor joins the queue first is $\mathbb{P}(X_1 > X_2) = \frac{\lambda_1}{\lambda_1 + \lambda_2},$ and one minus that for the probability that a sensor breaks and leaves the queue. We can then model the process by waiting $\text{Exp}(\lambda_1+\lambda_2)$ amount of time, and with probability $\frac{\lambda_1}{\lambda_1 + \lambda_2},$ adding a new sensor to the queue, or otherwise removing the oldest sensor from the queue.

## Dynamic Network Analysis
The continuous time model allows us to subsample as finely as we want. We keep track of the birth/death of the sensors in an interval tree; for any $t \in [0,T]$ we query the interval tree for the set of points that are present on the sphere at that time, and then create a dynamic network via the delaunay construction, assigining node values via some observation function representing temperature, or another desired quantity, and the edge weights by distance on the sphere.

In [23]:
import intervaltree as it

def get_sensor_lifetimes(time, birth_rate, death_rate):
    '''
    A shoddy implementation of a queueing model, with exponentially distributed processing time and
    exponentially distributed growth.
    '''
    l1 = 1/birth_rate
    l2 = 1/death_rate

    births = [0]*100 #initialize 100 points

    current_time = 0
    intervals = it.IntervalTree()

    while current_time < time:

        # if there are no current points
        if len(births) == 0:
            current_time += np.random.exponential(l2)
            births.append(current_time)
            continue

        current_time += np.random.exponential(1/(birth_rate + death_rate))

        # add a new point
        if np.random.rand() < l1/(l1 + l2):
            births.append(current_time)
        else:
            intervals[births.pop(0):current_time] = sample_uniform_sphere(1).tolist()[0]

    #add the remaining nodes:
    for i in range(len(births[:-1])):
        intervals[births[i]:time] = sample_uniform_sphere(1).tolist()[0]

    return intervals


def create_dynamic_networks(intervals,lambda1,lambda2, obs_times, observation_function, edge_wtsfn):
    '''
    Can query the interval tree below for more information about the points
    -----------------------------------------------------------------------
    Input:
    intervals - the birth deaths of the sensors
    lambda1 - birth rate (we expect to see this number born in a time interval
        of length 1)
    lambda2 - death rate (length of a lifetime of a node)
    obs_times - list of times at which to sample obs function
    observation_function - function from which to sample
    edge_wtsfn - function applied to edge wts
    '''

    ### query tree at each timestep ###
    coordinate_set = []
    for t in obs_times:
        points = intervals.at(t)
        coordinates = [ p[2] for p in list(points) ]

        coordinate_set.append(coordinates)

        hull = sp.ConvexHull(coordinates)
        node_wts.append(get_node_wts(t,hull,observation_function))
        edge_wts.append(edge_wtsfn(hull))

    return (node_wts,edge_wts,coordinate_set)



### Network Creation Code

In [10]:
def sample_uniform_sphere(N):
    cds = np.random.normal(0,1,(N,3))
    normalized_cds = cds/np.reshape(np.sqrt(np.sum(cds**2,axis = 1)),(N,1))
    return normalized_cds

def get_node_wts(t,hull_obj, obsfn):
    hull = hull_obj
    node_wts = [obsfn(t,p) for p in hull.points]
    return np.array(node_wts)

def get_edge_wts(hull_obj, alpha = 1.0):
    """
    map edges to their distances
    use this to create a dynamic network, edges are these distances.
    Returns
    -------
    edges: scipy.sparse(N, N)
        A sparse matrix with the edge weights
    alpha: float
        Amount by which to weight distances
    """
    hull = hull_obj
    edges = simplex_list_to_edge_list(hull.simplices)

    v = hull.points
    ds = [cartesian_to_sphere_distance(v[e[0],:], v[e[1],:]) for e in edges]
    ds = alpha*np.array(ds + ds)

    e0 = np.array([e[0] for e in edges] + [e[1] for e in edges])
    e1 = np.array([e[1] for e in edges] + [e[0] for e in edges])

    return sparse.coo_matrix((ds, (e0, e1)), shape=(len(v), len(v)))

In [11]:
def inverse_x_phi_fn(x):
    1.0/(a+0.1)
    return np.array([1/(a + 0.1) for a in x])

def linear_phi_fn(x):
    return 10-x

def identity_phi_fn(x):
    return x

In [4]:
# get the adjacency graph of this triangulation
import itertools as itr

def simplex_list_to_adjacency_graph(simplex_list):
    pass

# simplex_list is an array of arrays
def simplex_list_to_edge_list(simplex_list):
    edges = set()
    for simplex in simplex_list:
        simp_edges = set(itr.combinations(simplex,2)) 
        edges = edges.union(simp_edges)
    return edges

In [5]:
# cds should be an array of size (3,)
def cartesian_to_spherical(cds):
    (x,y,z) = cds
    rho = np.sqrt(np.sum(cds**2))
    r = np.sqrt(x**2 + y**2)
    theta = np.arccos(x/r)
    phi = np.arcsin(r/rho)
    
    return (rho,phi,theta)

def cartesian_to_sphere_distance(cds1,cds2):
    (rho,lat1,lon1) = cartesian_to_spherical(cds1)
    (rho,lat2,lon2) = cartesian_to_spherical(cds2)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = np.sqrt(np.sin(dlat/2)**2 + np.cos(lat2)*np.cos(lat1)*np.sin(dlon/2)**2)
    c = np.arcsin(a)
    
    return 2*rho*c

### Observation Function Code

In [3]:
# node weights are sampled from a periodic function on the sphere. sin(t + 2*pi*z)?
def periodic_northsouth_modulated(t,cds, T):
    """
    Parameters
    ----------
    t: float
        Time index
    cds: ndarray(N, 3)
        Cartesion coordinates of sphere points
    T: float
        Period of a cycle
    """
    rho, phi, theta = cartesian_to_spherical(cds)
    return 3 + 2*np.cos(2*np.pi*t/T + 2*phi)

### Evaluation Code

In [8]:
def get_maximum_persistence(PD):
    num_dim = len(PD)

    #has to be a 2D array
    def max_pers(array): 
        if len(array) == 0:
            return 0
        diff = array[:,1] - array[:,0]
        return max(diff)

    return list(map(max_pers,PD))

# gets the difference between the persistences of the first and second top features
def get_top_diff_persistence(PD):
    num_dim = len(PD)
    
    def get_diff(array):
        if len(array) == 0:
            return 0
        diff = array[:,1] - array[:,0]
        sorted_diff = sorted(diff,reverse = True)
        if len(sorted_diff) == 1:
            return sorted_diff[0]
        else:
            return sorted_diff[0] - sorted_diff[1]
        
    return(list(map(get_diff,PD)))
        
def get_num_features(PD):
    return(list(map(len,PD)))

### Pipeline Code

In [9]:
def apply_pipeline(node_wts, edge_wts, d, tau):
    
    # apply phi functions, and scale the weights
    phi_node_wts, phi_edge_wts = gf.weight_fn(node_wts, edge_wts, lamda=1, phi=identity_phi_fn)

    # constrcut the filtrations / simplicial complexes according to our construction
    filtration_matrix = list(map(lambda n, e: pf.get_filtration(n, e), phi_node_wts, phi_edge_wts))
    # summarize these filtrations using H_0 barcodes
    barcodes = list(map(pf.get_rips_complex, filtration_matrix))

    # get bottleneck distance between all the H0 diagrams;
    bn_dist_matrix = pf.get_bottleneck_dist_matrix(barcodes)
    
    # construct a sliding window embedding
    sw_vecs_indices = sw.sliding_window(range(len(barcodes)), d=d, tau=tau)
    sw_dist_matrix = sw.sw_distance_matrix(sw_vecs_indices, bn_dist_matrix)

    # get H1 from the sliding window distance matrix
    PDs = ripser(sw_dist_matrix, distance_matrix=True, maxdim=1, coeff=2)['dgms']
    
    return PDs

# Run the Dynamic Network Analysis

In [6]:
from __future__ import division
import numpy as np
import sys
import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial.distance import squareform
sys.path.append('../shared_scripts/')
import graph_fns as gf
import persistence_fns as pf
import sliding_window_fns as sw
from ripser import ripser, plot_dgms
from sklearn import manifold

In [24]:
## Setup Sensor Lifetimes
T = 10 # Period 
dynamic_network_samples = 200
ts = np.arange(0,2*T,dynamic_network_samples)

obsfn = lambda t, p: periodic_northsouth_modulated(t,p,T)
edge_wtsfn = lambda hull_obj: get_edge_wts(hull_obj, alpha = 1.0)

lambda1 = 20
lambda2 = lambda1

sensor_lifetimes = get_sensor_lifetimes(2*T, lambda1, lambda2)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [22]:
sensor_lifetimes.items()

{Interval(0, 0.1520925586639896, array([[-0.12094897,  0.17226228, -0.97759759]])),
 Interval(0, 0.3024989559738058, array([[ 0.11864593,  0.5281908 ,  0.84079583]])),
 Interval(0, 0.30620861211746675, array([[ 0.62395368, -0.52045002,  0.58293531]])),
 Interval(0, 0.5231641742172616, array([[ 0.25095724, -0.44619535,  0.85902862]])),
 Interval(0, 0.6702574037986431, array([[ 0.5819328 ,  0.52720188, -0.61920304]])),
 Interval(0, 0.9702631974524337, array([[-0.68939517,  0.67414986,  0.26505899]])),
 Interval(0, 1.1203693789929117, array([[ 0.47465645,  0.82775032, -0.29921674]])),
 Interval(0, 1.1525125850391735, array([[-0.0294004 ,  0.91461986, -0.40324451]])),
 Interval(0, 1.2195717772459587, array([[ 0.18933784, -0.02507746, -0.98159172]])),
 Interval(0, 1.317559990364011, array([[-0.6889888 , -0.69934118,  0.19030592]])),
 Interval(0, 1.4878861386930562, array([[ 0.78483661, -0.15439324,  0.60016184]])),
 Interval(0, 1.6070049361792076, array([[-0.44939356, -0.28213811,  0.847610

In [ ]:
## Create the Dynamic Network
(node_wts,edge_wts, allpoints) = create_dynamic_network(ts, obsfn=obsfn,
                                    edge_wtsfn=edge_wtsfn, change_param=change_param)